In [20]:
import sys

sys.path.append("../")
from calculus_path_mod.term_engine import *
from calculus_path_mod.reduction_strategy import *
from calculus_path_mod.json_serialization import save_terms

Lambda = Abstraction
App = Application

# <span style="color: #A1F99C">USEFUL TERMS</span>

In [21]:
def ite_term():
    x, y, c = Var(), Var(), Var()
    x_, y_, c_ = Atom(x), Atom(y), Atom(c)
    return Lambda(c, Lambda(x, Lambda(y, multi_app_term(c_, x_, y_))))


def true_term():
    x, y = Var(), Var()
    x_ = Atom(x)
    return Lambda(x, Lambda(y, x_))


def num_zero_term():
    s, z = Var(), Var()
    z_ = Atom(z)
    return Lambda(s, Lambda(z, z_))


def num_term(n: int):
    if n <= 0:
        return num_zero_term()
    s, z = Var(), Var()
    s_, z_ = Atom(s), Atom(z)
    core_term = App(s_, z_)
    for _ in range(n - 1):
        core_term = App(s_, core_term)
    return Lambda(s, Lambda(z, core_term))


def multi_app_term(term_0: Term, term_1: Term, *terms: Term):
    res_app_term = App(term_0, term_1)
    for term in terms:
        res_app_term = App(res_app_term, term)
    return res_app_term


def false_term():
    x, y = Var(), Var()
    y_ = Atom(y)
    return Lambda(x, Lambda(y, y_))


def pair_term():
    x, y, p = Var(), Var(), Var()
    x_, y_, p_ = Atom(x), Atom(y), Atom(p)
    return Lambda(x, Lambda(y, Lambda(p, multi_app_term(p_, x_, y_))))


def first_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, true_term()))


def second_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, false_term()))


def succ_term():
    x, y, n = Var(), Var(), Var()
    x_, y_, n_ = Atom(x), Atom(y), Atom(n)
    return Lambda(n, Lambda(x, Lambda(y, App(x_, multi_app_term(n_, x_, y_)))))


def sinc_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(
        p,
        multi_app_term(
            pair_term(),
            App(second_term(), p_),
            App(succ_term(), App(second_term(), p_)),
        ),
    )


def pred_term():
    n, f, x, g, h, u = Var(), Var(), Var(), Var(), Var(), Var()
    n_, f_, x_ = Atom(n), Atom(f), Atom(x)
    g_, h_, u_ = Atom(g), Atom(h), Atom(u)

    return Lambda(n, Lambda(f, Lambda(x, multi_app_term(
        n_,
        Lambda(g, Lambda(h, App(h_, App(g_, f_)))),
        Lambda(u, x_),
        Lambda(u, u_)
    ))))


def plus_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(m, Lambda(n, multi_app_term(n_, succ_term(), m_)))


def subtract_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(n, Lambda(m, multi_app_term(m_, pred_term(), n_)))


def iszero_term():
    x, n = Var(), Var()
    n_ = Atom(n)
    return Lambda(n, App(App(n_, Lambda(x, false_term())), true_term()))


def leq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n, Lambda(m, App(iszero_term(), multi_app_term(subtract_term(), n_, m_)))
    )


def and_term():
    a, b = Var(), Var()
    a_, b_ = Atom(a), Atom(b)
    return Lambda(a, Lambda(b, multi_app_term(ite_term(), a_, b_, a_)))


def eq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n,
        Lambda(
            m,
            multi_app_term(
                and_term(),
                multi_app_term(leq_term(), n_, m_),
                multi_app_term(leq_term(), m_, n_),
            ),
        ),
    )

# <span style="color: #A1F99C">1.</span>
<span style="color:#bb91f3">(λx. (λy.y)) ((λx. (xx)) (λx. (xx)))</span> == <span style="color:#f1bd47">I</span> == <span style="color:#f1bd47">λx. x</span>
<span style="color:#c9fafa">This term reduce in one step by LO, RO strategy
And reduce in the same terms with other strategy.</span>

In [22]:
def term_test_1():
    x, y = Var(), Var()
    x_, y_ = Atom(x), Atom(y)

    expected_term = Lambda(x, x_)
    return App(
        Lambda(x, Lambda(y, y_)),
        App(
            Lambda(x, App(x_, x_)),
            Lambda(x, App(x_, x_))
        )
    ), expected_term

# <span style="color: #A1F99C">2.</span>
<span style="color:#bb91f3">(ITE TRUE 3 1)</span> == <span style="color:#f1bd47>">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [23]:
def term_test_2():
    expected_term = num_term(3)
    return multi_app_term(ite_term(), true_term(), num_term(3), num_term(1)), expected_term

# <span style="color: #A1F99C">3.</span>
<span style="color:#bb91f3">(ITE FALSE 0 4)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [24]:
def term_test_3():
    expected_term = num_term(4)
    return multi_app_term(ite_term(), false_term(), num_term(0), num_term(4)), expected_term

# <span style="color: #A1F99C">4.</span>
<span style="color:#bb91f3">(PRED 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [25]:
def term_test_4():
    expected_term = num_term(0)
    return multi_app_term(pred_term(), num_term(0)), expected_term

# <span style="color: #A1F99C">5.</span>
<span style="color:#bb91f3">(PRED 5)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [26]:
def term_test_5():
    expected_term = num_term(4)
    return multi_app_term(pred_term(), num_term(5)), expected_term

# <span style="color: #A1F99C">6.</span>
<span style="color:#bb91f3">(PLUS 0 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [27]:
def term_test_6():
    expected_term = num_term(0)
    return multi_app_term(plus_term(), num_term(0), num_term(0)), expected_term

# <span style="color: #A1F99C">7.</span>
<span style="color:#bb91f3">(PLUS 2 3)</span> == <span style="color:#f1bd47">5</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x (x y)))))</span>

In [28]:
def term_test_7():
    expected_term = num_term(5)
    return multi_app_term(plus_term(), num_term(2), num_term(3)), expected_term

# <span style="color: #A1F99C">8.</span>
<span style="color:#bb91f3">(SUBTRACT 4 1)</span> == <span style="color:#f1bd47">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [29]:
def term_test_8():
    expected_term = num_term(3)
    return multi_app_term(subtract_term(), num_term(4), num_term(1)), expected_term

# <span style="color: #A1F99C">9.</span>
<span style="color:#bb91f3">(SUBTRACT 3 5)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [30]:
def term_test_9():
    expected_term = num_term(0)
    return multi_app_term(subtract_term(), num_term(3), num_term(5)), expected_term

# <span style="color: #A1F99C">10.</span>
<span style="color:#bb91f3">(LEQ 3 2)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [31]:
def term_test_10():
    expected_term = false_term()
    return multi_app_term(leq_term(), num_term(3), num_term(2)), expected_term

# <span style="color: #A1F99C">11.</span>
<span style="color:#bb91f3">(LEQ 2 5)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [32]:
def term_test_11():
    expected_term = true_term()
    return multi_app_term(leq_term(), num_term(2), num_term(5)), expected_term

# <span style="color: #A1F99C">12.</span>
<span style="color:#bb91f3">(EQ 2 2)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [33]:
def term_test_12():
    expected_term = true_term()
    return multi_app_term(eq_term(), num_term(2), num_term(2)), expected_term

# <span style="color: #A1F99C">13.</span>
<span style="color:#bb91f3">(EQ 1 4)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [34]:
def term_test_13():
    expected_term = false_term()
    return multi_app_term(eq_term(), num_term(1), num_term(4)), expected_term

# Save terms and expected normalized terms to the files

In [35]:
list_test_terms = [
    term_test_1(),
    term_test_2(),
    term_test_3(),
    term_test_4(),
    term_test_5(),
    term_test_6(),
    term_test_7(),
    term_test_8(),
    term_test_9(),
    term_test_10(),
    term_test_11(),
    term_test_12(),
    term_test_13(),
]

In [36]:
# save_terms("terms_src/test_terms.dat", [tt[0] for tt in list_test_terms])
# save_terms("terms_src/expected_terms.dat", [tt[1] for tt in list_test_terms])

In [47]:
tt1, et1 = list_test_terms[9]
nt1, _ = tt1.normalize(LOStrategy())
print(tt1.funky_str())
print(nt1.funky_str())
print(et1.funky_str())

(((λx.(λy.((λb.((b (λa.(λc.(λd.d)))) (λe.(λj.e)))) (((λi.(λn.((n (λm.(λt.(λr.(((m (λq.(λw.(w (q t))))) (λu.r)) (λu.u)))))) i))) x) y)))) (λo.(λp.(o (o (o p)))))) (λs.(λf.(s (s f)))))
(λy.(λx.x))
(λx.(λy.y))


In [48]:
et1.update_vars_order()
nt1.update_vars_order()

print(nt1.funky_str())
print(et1.funky_str())

{5600: 9089, 5601: 9090}
{9087: 9091, 9088: 9092}
(λx.(λy.y))
(λx.(λy.y))


In [49]:
"(λx.(λy.(x (x (x y)))))" == "(λx.(λy.(x (x (x y)))))"

True